In [ ]:
# checking GPU
!nvidia-smi

Tue Apr  5 18:05:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Configure to use GPU and CUDNN
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# compile
!make

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Change lines in yolov3.cfg file
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
# number of classes * 2000
!sed -i 's/max_batches = 500200/max_batches = 2000/' cfg/yolov3_training.cfg
# number of classes
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
# (5 + number of classes) * 3
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Store object detail
!echo -e 'liscense_plate' > data/obj.names

In [ ]:
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /content/gdrive/MyDrive/yolov3' > data/obj.data


In [ ]:
!cp cfg/yolov3_training.cfg /content/gdrive/MyDrive/yolov3/yolov3_testing.cfg
!cp data/obj.names /content/gdrive/MyDrive/yolov3/classes.txt


In [ ]:
!mkdir /content/darknet/data/obj
!unzip /content/gdrive/MyDrive/yolov3/images.zip -d /content/darknet/data/obj

In [ ]:
import glob
images_list = glob.glob("/content/darknet/data/obj/*.jpg")
with open("/content/darknet/data/train.txt", "w") as f:
    f.write("\n".join(images_list))

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# re-start your training from last saved weights
!./darknet detector train data/obj.data cfg/yolov3_training.cfg  /content/gdrive/MyDrive/yolov3/yolov3_training_last.weights -dont_show